In [2]:
import re
from rapidfuzz import fuzz
from helper import load_json_file, save_json_file
# from doctr.io import DocumentFile
# from doctr.models import ocr_predictor

In [3]:
# model = ocr_predictor(pretrained=True)
# doc = DocumentFile.from_pdf("sample/mandiri-transaction_inquery.pdf")
# result = model(doc)

# save_json_file("sample/mandiri-transaction_inquery.json", result.export())

In [43]:
data = load_json_file("sample/mandiri-account_statement_report.json")
settings = load_json_file("sample/setting_parameter.json")

In [5]:
def normalize_string(s):
    return re.sub(r'\W+', ' ', s).lower()

In [47]:
def get_statement_id(data):
    normalized_data = normalize_string(data)
    
    ranking = []
    
    for setting in settings:
        total_word_count = sum(len(keyword.split()) for keyword in setting['keywords'])
        
        scores = [fuzz.partial_ratio(normalize_string(keyword), normalized_data)
                  for keyword in setting['keywords'] if fuzz.partial_ratio(normalize_string(keyword), normalized_data) > 80]  
        
        if scores:
            final_score = sum(scores) * total_word_count
            ranking.append({
                "setting": setting,
                "final_score": final_score
            })

    if not ranking:
        return None
    
    return max(ranking, key=lambda x: x["final_score"])["setting"]["id"]

In [83]:
def table_content(sentence, statement_id=0):
    upper_limit = normalize_string(' '.join(filter(None, settings[statement_id]['column_names'].values())))
    lower_limit = normalize_string(settings[statement_id]['buttom_devider'])
    
    threshold = 80

    upper_start_index = 0  
    lower_end_index = len(sentence) 

    if fuzz.partial_ratio(upper_limit, sentence) >= threshold:
        upper_start_index = sentence.find(upper_limit) + len(upper_limit)

    if fuzz.partial_ratio(lower_limit, sentence) >= threshold:
        lower_end_index = sentence.find(lower_limit)

    cut_sentence_output = sentence[upper_start_index:lower_end_index].strip()
    
    print(cut_sentence_output)


test_sentance = 'laporan rekening koran a a  account statement report  mandiri account no 16500 idr surya agung period 01 aug 2022  31 aug 2022 currency idr branch kcjkt gedung pusat kehutanan opening balance 1 669 379 38 posting date remark reference no debit credit balance inv49tbs inv491bs 01 08 2022 14 30  sap inv vabe tbs 04 mcm inhousetri dari inv49 tbs sap 0 00 200 000 000 00 201 669 379 38 sebaungsawit plantations99102 01 08 2022 14 30  inv mcm in220858po231 ainhousetrf ke etam 12 337 102 00 33 0 00 189 332 277 38 wira utama99102 0408202 14 30  sutam ke 15 000 000 00 0 00 174 332 277 38 herni99102 c200202 15 22  mcm inhousetri ke the 7 000 000 00 0 00 167 332 277 38 herlnafranci559102 02 08 2022 15 22  tiket mcm inhousetrf ke beni 27 togar siregar99102 3 900016 00 0 00 163 423 861 38 02 08 2022 15 26  ops  mcm jul ainhousett ke imron 24 3 42720000 0 00 159 996 761 38 subhan99102 02 08 2022 15 26  ops  mcm ainhousetri lemburjul ke andi 156 049 461 38 27 3 947300 00 0 00 gunawan99102 02 08 2022 15 27  mcm vinny 27 iohoancnunrpr 840 00000 0 00 155 209 461 38 surya agung prima 15 27  mcm outwcn 0 00 0302024 sutarti99102  surya 840 00000 154 369 461 38 agungprima 02 08 2022 15 27  mcmoutw milki silmi 0 00 27 ngmomnicn surya 040 000 153 529 461 38 agung prima 0202021 15 27  clearingfee clearing feo 99102 2 90000 0 00 153 526 561 38 0206202 15 27  clearing clearing fee fee 99102 2 900 00 0 00 153 523 661 38 02 08 2022 15 27  clearing fee 0 00 27 clearing fee 99102 2 900 00 153 520 761 38 02 08 2022 30 18 49  pengembaliantket online tri 152137 1 981 300 00 0 00 151 539 361 38 53940101445750699102 02 08 2022 18 49  transferfee 0 00 151 532 861 38 30 99102 55000 00202 18 50  ke 5 486 000 00 0 00 146 048 861 38 02 08 2022 46 18 50  ke 3 557 000 00 0 00 142 489 861 38 le eofauis 02 08 2022 50 18 50  mcminhousett ke 830 000 00 0 00 141 659 861 38 sunarti99102 02 08 2022 18 50  mcminhousetr po367 ke 2 700 000 00 0 00 138 959 861 38 54 maryanto99102 02 08 2022 18 50  po368 mcminhousetrf ke yenny 129 000 00 0 00 138 830 861 38 57 natalia99102 02 08 2022 01 18 51  rostpnoete ke yulia 20 000 00 0 00 138 810 861 38 0206202 18 52  152164 108 000 00 0 00 138 702 861 38 02 08 2022 11 18 52  6 3000 0 00 138 696 361 38 transfer fee 99102 le 02 08 2022 18 52  po371  mcm inhouseti 372 ke yossy 151 000 00 0 00 138 545 361 38 11 lingga99102 '
table_content(test_sentance, 0)

inv49tbs inv491bs 01 08 2022 14 30  sap inv vabe tbs 04 mcm inhousetri dari inv49 tbs sap 0 00 200 000 000 00 201 669 379 38 sebaungsawit plantations99102 01 08 2022 14 30  inv mcm in220858po231 ainhousetrf ke etam 12 337 102 00 33 0 00 189 332 277 38 wira utama99102 0408202 14 30  sutam ke 15 000 000 00 0 00 174 332 277 38 herni99102 c200202 15 22  mcm inhousetri ke the 7 000 000 00 0 00 167 332 277 38 herlnafranci559102 02 08 2022 15 22  tiket mcm inhousetrf ke beni 27 togar siregar99102 3 900016 00 0 00 163 423 861 38 02 08 2022 15 26  ops  mcm jul ainhousett ke imron 24 3 42720000 0 00 159 996 761 38 subhan99102 02 08 2022 15 26  ops  mcm ainhousetri lemburjul ke andi 156 049 461 38 27 3 947300 00 0 00 gunawan99102 02 08 2022 15 27  mcm vinny 27 iohoancnunrpr 840 00000 0 00 155 209 461 38 surya agung prima 15 27  mcm outwcn 0 00 0302024 sutarti99102  surya 840 00000 154 369 461 38 agungprima 02 08 2022 15 27  mcmoutw milki silmi 0 00 27 ngmomnicn surya 040 000 153 529 461 38 agung 

In [1]:
def get_table(data):
    result = []
    for page in data["pages"]:
        id = page["page_idx"]
        combined_text = ""
        for block in page['blocks']:
            for line in block['lines']:
                for word in line['words']:
                    combined_text += normalize_string(word['value']) + " "
        
        statement_id = get_statement_id(combined_text) 

        normalized_page = {
                "id": id,
                "statement_id": statement_id
            }
        
        result.append(normalized_page)  
               
    return result

result = get_table(data)

print(result[0])

NameError: name 'data' is not defined

In [7]:
def normalize_json(data):
    result = {"pages": []}

    for page in data['pages']:
        page_id = page['page_idx']
        word_entries = []
        combined_words = []

        for block in page['blocks']:
            for line in block['lines']:
                for word in line['words']:
                    if len(combined_words) < 12:
                        combined_words.append(normalize_string(word['value']))
                    word_entry = {
                        "value": word['value'],
                        "confidence": word['confidence'],
                        "geometry": word['geometry']
                    }
                    word_entries.append(word_entry)
        
        cleaned_words = [''.join(word.split()) for word in combined_words]
        combined_words_text = ' '.join(cleaned_words)

        statement_type = get_statement_type(combined_words_text)
        
        normalized_page = {
            "page_id": page_id,
            "statement": statement_type,
            "words": word_entries
        }

        result["pages"].append(normalized_page)
    
    return result


In [8]:
normalize_result = normalize_json(data)
save_json_file("sample/mandiri-transaction_inquery_normalized.json", normalize_result)

In [9]:
def find_header_coordinate(data):
    column_names = list(filter(None, data['statement']['column_names'].values()))

    for word in data['words']:
        for column_name in column_names:
            if fuzz.partial_ratio(normalize_string(column_name), normalize_string(word['value'])) > 80:
                print(word['value'], word['geometry'])


test_page = normalize_result['pages'][0]
transaction_result = find_header_coordinate(test_page)


a [[0.6920923056722689, 0.0224609375], [0.701766018907563, 0.0283203125]]
a [[0.6920923056722689, 0.0224609375], [0.701766018907563, 0.0283203125]]
a [[0.6920923056722689, 0.0224609375], [0.701766018907563, 0.0283203125]]
A [[0.7197314863445379, 0.0205078125], [0.739078912815126, 0.0302734375]]
A [[0.7197314863445379, 0.0205078125], [0.739078912815126, 0.0302734375]]
A [[0.7197314863445379, 0.0205078125], [0.739078912815126, 0.0302734375]]
No [[0.2070246848739496, 0.06640625], [0.22775407037815126, 0.078125]]
Balance [[0.20840664390756303, 0.1328125], [0.25539325105042016, 0.1435546875]]
Posting [[0.1655659138655462, 0.1865234375], [0.21393448004201682, 0.201171875]]
Date [[0.2111705619747899, 0.1865234375], [0.24571953781512607, 0.2001953125]]
Remark [[0.2636850052521008, 0.1865234375], [0.31481748949579835, 0.2001953125]]
Reference [[0.4018809086134454, 0.181640625], [0.4668329831932773, 0.1943359375]]
No [[0.4018809086134454, 0.193359375], [0.4239922531512605, 0.2041015625]]
Debit [